In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
import numpy as np

file = 'melbourne_housing_raw.csv'
data = pd.read_csv(file)

cleaned_data = data.dropna(subset=['Price'])

X = cleaned_data.drop(['Price', 'Suburb', 'Date', 'CouncilArea', 'Regionname', 'SellerG', 'Type', 'Method'], axis=1)
y = cleaned_data['Price']

imputer = SimpleImputer(strategy='mean')
X_filled = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X_filled, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(random_state=42)

rf_model.fit(X_train, y_train)
initial_pred = rf_model.predict(X_test)
initial_mae = mean_absolute_error(y_test, initial_pred)

mae_values = [initial_mae]
features_list = X_train.columns.tolist()

for i in range(len(features_list) - 1):
    rf_model.fit(X_train, y_train)
    importances = rf_model.feature_importances_

    least_important_idx = np.argmin(importances)
    least_important_feat = features_list[least_important_idx]

    X_train = X_train.drop(columns=[least_important_feat])
    X_test = X_test.drop(columns=[least_important_feat])

    rf_model.fit(X_train, y_train)
    predictions = rf_model.predict(X_test)

    mae_value = mean_absolute_error(y_test, predictions)
    mae_values.append(mae_value)

    features_list.remove(least_important_feat)

mae_values



[179476.4312564365,
 179550.99279074298,
 180086.93622940636,
 183336.16241149075,
 184117.91730353973,
 185062.34566818757,
 192390.0496203936,
 198422.50520947954,
 207475.50484132508,
 229791.3428620935,
 335183.8556700821,
 336796.30023891194]